In [1]:
import numpy as np
import cv2
import json
import os
from tqdm import tqdm
from itertools import combinations
from sklearn.metrics import pairwise


In [ ]:

json_dir='/content/drive/MyDrive/posedetection/output_json_new/'
final_data=np.array([])
filenames=[]
for filename in os.listdir(json_dir):
  print(filename)
  filenames.append([filename][0][:-15])

  f = open(json_dir+filename,)
  data = json.load(f)
  keypoints=list(data.items())[1][1][0]['pose_keypoints_2d']
  image_keypoints=[]
  # image_keypoints=np.append(image_keypoints,np.array([filename][0][:-15]))
  
  for keys in range(0,len(keypoints),3):
    image_keypoints.append(np.array([keypoints[keys],keypoints[keys+1],keypoints[keys+2]]))
  final_data=np.append(final_data,image_keypoints)
  print(final_data.shape)

  f.close()
# final_data=final_data.reshape(15,36)

movingbackward_2_keypoints.json
(54,)
movingbackward_1_keypoints.json
(108,)
movingbackward_keypoints.json
(162,)
endingpose_2_keypoints.json
(216,)
endingpose_1_keypoints.json
(270,)
endingpose_keypoints.json
(324,)
completesquat_2_keypoints.json
(378,)
completesquat_1_keypoints.json
(432,)
completesquat_keypoints.json
(486,)
startingpose_2_keypoints.json
(540,)
startingpose_keypoints.json
(594,)
startingpose_1_keypoints.json
(648,)
movingforward_2_keypoints.json
(702,)
movingforward_keypoints.json
(756,)
movingforward_1_keypoints.json
(810,)


In [ ]:
x=final_data.reshape(15,18,3)

In [ ]:
x.shape

(15, 18, 3)

In [ ]:
filenames

['movingbackward_2',
 'movingbackward_1',
 'movingbackward',
 'endingpose_2',
 'endingpose_1',
 'endingpose',
 'completesquat_2',
 'completesquat_1',
 'completesquat',
 'startingpose_2',
 'startingpose',
 'startingpose_1',
 'movingforward_2',
 'movingforward',
 'movingforward_1']

In [ ]:
with open('test.npy', 'wb') as f:
  np.save(f,x)

In [ ]:
def scale_transform_normalize(coords):
    """
    Parameters:
    coords (ndarray): array of (x,y,c) coordinates

    Returns:
    ndarray: coords scaled to 1x1 with center at (0,0)
    ndarray: confidence scores of each joint
    """
    coords, scores = coords[:,:,:-1], coords[:,:,-1]
    print(coords[:,:,-1].shape)
    diff = coords.max(axis=1) - coords.min(axis=1)
    diff_max = np.max(diff, axis=0)
    mean = coords.mean(axis=1).reshape(coords.shape[0],1,coords.shape[-1])
    out = (coords - mean) / diff_max
    
    return out, scores

In [ ]:
X=x
X.shape

(15, 18, 3)

In [ ]:
X_norm, scores = scale_transform_normalize(X)

(15, 18)


In [ ]:
N,D,C = X.shape
import time
# Prepare X
X_norm, scores = scale_transform_normalize(X)
scores = scores.reshape((N, D, 1))
# X_norm = np.concatenate([X_norm, scores], axis=2)
X_norm = X_norm.reshape((X_norm.shape[0], 1, -1))
X_norm /= np.linalg.norm(X_norm, axis=2)[:, :, np.newaxis]

res=[]

start = time.time()
for index in tqdm(combinations(np.arange(N), 2)):
    vec_1 = X_norm[index[0]]
    vec_2 = X_norm[index[1]]
    cosine_score = pairwise.cosine_similarity(vec_1, vec_2)[0]
    res.append([filenames[index[0]],filenames[index[1]], cosine_score[0]])
print('time taken to run script %.5f'% (time.time()-start))

105it [00:00, 5175.72it/s]

(15, 18)
time taken to run script 0.02248


In [ ]:
res

[['movingbackward_2', 'movingbackward_1', 0.999639909160638],
 ['movingbackward_2', 'movingbackward', 0.9996400303610741],
 ['movingbackward_2', 'endingpose_2', 0.9809150888116722],
 ['movingbackward_2', 'endingpose_1', 0.9798616467588571],
 ['movingbackward_2', 'endingpose', 0.9800576893546712],
 ['movingbackward_2', 'completesquat_2', 0.8399640009535756],
 ['movingbackward_2', 'completesquat_1', 0.8414782398454315],
 ['movingbackward_2', 'completesquat', 0.8415158488039012],
 ['movingbackward_2', 'startingpose_2', 0.9808444431098067],
 ['movingbackward_2', 'startingpose', 0.9809171363613466],
 ['movingbackward_2', 'startingpose_1', 0.9806863060222997],
 ['movingbackward_2', 'movingforward_2', 0.6006992505660894],
 ['movingbackward_2', 'movingforward', 0.717980771777413],
 ['movingbackward_2', 'movingforward_1', 0.6000609851589966],
 ['movingbackward_1', 'movingbackward', 0.9999999998831794],
 ['movingbackward_1', 'endingpose_2', 0.9842479810949442],
 ['movingbackward_1', 'endingpose_

In [ ]:
import pandas as pd
df=pd.DataFrame(res,columns=['user_pose','pose_to_match','score'])


In [ ]:
df.sort_values(['score'],ascending=False)

,user_pose,pose_to_match,score
14,movingbackward_1,movingbackward,1.000000
77,completesquat_1,completesquat,0.999997
91,startingpose_2,startingpose_1,0.999982
95,startingpose,startingpose_1,0.999976
90,startingpose_2,startingpose,0.999964
...,...,...,...
99,startingpose_1,movingforward_2,0.597677
98,startingpose,movingforward_1,0.596948
101,startingpose_1,movingforward_1,0.596199
47,endingpose_2,movingforward_2,0.595304


In [ ]:
df[df['user_pose']=='startingpose_1'].sort_values(['score'],ascending=False)

,user_pose,pose_to_match,score
100,startingpose_1,movingforward,0.707422
99,startingpose_1,movingforward_2,0.597677
101,startingpose_1,movingforward_1,0.596199


In [ ]:
df[df['pose_to_match']=='startingpose_1'].sort_values(['score'],ascending=False)

,user_pose,pose_to_match,score
91,startingpose_2,startingpose_1,0.999982
95,startingpose,startingpose_1,0.999976
56,endingpose_1,startingpose_1,0.999536
65,endingpose,startingpose_1,0.999502
46,endingpose_2,startingpose_1,0.996027
35,movingbackward,startingpose_1,0.984381
23,movingbackward_1,startingpose_1,0.984381
10,movingbackward_2,startingpose_1,0.980686
86,completesquat,startingpose_1,0.879148
80,completesquat_1,startingpose_1,0.879038
